# 1、项目介绍
* 介绍：首先这个项目是实验楼上的一个免费项目，[原项目链接](https://www.shiyanlou.com/courses/599)在这里。该项目主要涉及到爬虫、高德API的使用，用到的包主要有requests 、BeautifulSoup、csv。
* 功能：本项目使用 Python 脚本爬取某租房网站的房源信息，利用高德的 js API 在地图上标出房源地点，划出距离工作地点1小时内可到达的范围。
* 环境：实验楼的实验是在Linux环境下进行的，这里我使用的是windows系统、Pycharm IDE。

# 2、具体流程
## 2.1 软件架构
* 首先打开PyCharm软件，在某个路径下新建项目（该路径后面还会用到），然后新建一个scrawl.py文件，该文件主要用于爬去某租房网站的房源信息，并把信息存储到rent.csv文件中。然后在新建一个index.html文件，用于高德地图页面设计。具体架构如图1所示：<img src="1.png">
## 2.2爬虫代码
* 接下来就是代码得到编写，首先记得在  File | Settings | Project: RentHouse | Project Interpreter  中添加相应的库噢！
* 注：自己一开始完全按照实验楼中的代码，但是后来有问题，自己对有些地方做了小的改动，会在代码中具体标出。
* scrawl.py具体代码如下


In [ ]:
import  csv
from bs4 import BeautifulSoup
import requests
url = "https://bj.58.com/pinpaigongyu/pn/{page}/?minprice=3000_4000"
page = 0
'''下面这句做了修改，添加了newline="",encoding='utf-8'，如果指定编码，后面在地图中添加房源信息时会失败
（不信可以试试呀，哈哈！自己花了好久才从坑里爬出来）'''
#csv_file = open("rent1.csv",'w')  原代码
csv_file = open("rent1.csv",'w',newline="",encoding='utf-8')
csv_writer = csv.writer(csv_file,delimiter=',')
while True:
    page += 1
    print("fetch: ",url.format(page = page))
    response = requests.get(url.format(page=page))
    html = BeautifulSoup(response.text,features="html.parser")
    house_list = html.select(".list > li")

    if not house_list:
        break
    for house in house_list:
        house_title = house.select("h2")[0].string
        house_url = "https://bj.58.com/%s"%(house.select("a")[0]["href"])
        house_info_list = house_title.split()
        if "公寓" in house_info_list[1] or "青年社区" in house_info_list[1]:
            house_location = house_info_list[0]
        else:
            house_location = house_info_list[1]
            house_money = house.select(".money")[0].select("b")[0].string
            csv_writer.writerow([house_title,house_location,house_money,house_url])
            # # csv_writer.writerow([house_title, house_location, house_url])
            # csv_writer.writerow([house_title, house_location])
csv_file.close()

* 开始运行爬虫的代码，爬...爬...效果如图2所示。
* 注意：有时候可能因为频繁的爬取，网站被您爬烦了😄。这时候你需要自己打开网站，会提示你滑动验证，验证好了再继续爬就好了。
* 图2<img src='2.png'>

## 2.2高德地图网页设计
* 这部分我自己本来也不太懂，但是看看[高德JavaScript API帮助文档](https://lbs.amap.com/api/javascript-api/summary/)、[高德JavaScript API示例中心](https://lbs.amap.com/demo-center/js-api)、[Requests:HTTP for Humans](http://www.python-requests.org/en/master/)、[Beautiful Soup 4.2.0文档](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)基本能有个大致的了解。
script.html文件代码如下：

In [2]:
<html>
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="initial-scale=1.0, user-scalable=no, width=device-width">
        <title>毕业生租房</title>
        <link rel="stylesheet" href="http://cache.amap.com/lbs/static/main1119.css" />
        <link rel="stylesheet" href="http://cache.amap.com/lbs/static/jquery.range.css" />
        <script src="http://cache.amap.com/lbs/static/jquery-1.9.1.js"></script>
        <script src="http://cache.amap.com/lbs/static/es5.min.js"></script>
        <script src="http://webapi.amap.com/maps?v=1.3&key=22d3816e107f199992666d6412fa0691&plugin=AMap.ArrivalRange,AMap.Scale,AMap.Geocoder,AMap.Transfer,AMap.Autocomplete"></script>
        <script src="http://cache.amap.com/lbs/static/jquery.range.js"></script>
        <style>
        .control-panel {
            position: absolute;
            top: 30px;
            right: 20px;
        }

        .control-entry {
            width: 280px;
            background-color: rgba(119, 136, 153, 0.8);
            font-family: fantasy, sans-serif;
            text-align: left;
            color: white;
            overflow: auto;
            padding: 10px;
            margin-bottom: 10px;
        }

        .control-input {
            margin-left: 120px;
        }

        .control-input input[type="text"] {
            width: 160px;
        }

        .control-panel label {
            float: left;
            width: 120px;
        }

        #transfer-panel {
            position: absolute;
            background-color: white;
            max-height: 80%;
            overflow-y: auto;
            top: 30px;
            left: 20px;
            width: 250px;
        }
        </style>
    </head>

    <body>
        <div id="container"></div>
        <div class="control-panel">
            <div class="control-entry">
                <label>选择工作地点：</label>
                <div class="control-input">
                    <input id="work-location" type="text">
                </div>
            </div>
            <div class="control-entry">
                <label>选择通勤方式：</label>
                <div class="control-input">
                    <input type="radio" name="vehicle" value="SUBWAY,BUS" onClick="takeBus(this)" checked/> 公交+地铁
                    <input type="radio" name="vehicle" value="SUBWAY" onClick="takeSubway(this)" /> 地铁
                </div>
            </div>
            <div class="control-entry">
                <label>导入房源文件：</label>
                <div class="control-input">
                    <input type="file" name="file" onChange="importRentInfo(this)" />
                </div>
            </div>
        </div>
        <div id="transfer-panel"></div>
        <script>
        var map = new AMap.Map("container", {
            resizeEnable: true,
            zoomEnable: true,
            center: [116.397428, 39.90923],
            zoom: 11
        });
        var scale = new AMap.Scale();
map.addControl(scale);

var arrivalRange = new AMap.ArrivalRange();
var x, y, t, vehicle = "SUBWAY,BUS";
var workAddress, workMarker;
var rentMarkerArray = [];
var polygonArray = [];
var amapTransfer;

var infoWindow = new AMap.InfoWindow({
    offset: new AMap.Pixel(0, -30)
});

var auto = new AMap.Autocomplete({
    input: "work-location"
});
AMap.event.addListener(auto, "select", workLocationSelected);


function takeBus(radio) {
    vehicle = radio.value;
    loadWorkLocation()
}

function takeSubway(radio) {
    vehicle = radio.value;
    loadWorkLocation()
}

function importRentInfo(fileInfo) {
    var file = fileInfo.files[0].name;
    loadRentLocationByFile(file);
}

function workLocationSelected(e) {
    workAddress = e.poi.name;
    loadWorkLocation();
}

function loadWorkMarker(x, y, locationName) {
    workMarker = new AMap.Marker({
        map: map,
        title: locationName,
        icon: 'http://webapi.amap.com/theme/v1.3/markers/n/mark_r.png',
        position: [x, y]

    });
}


function loadWorkRange(x, y, t, color, v) {
    arrivalRange.search([x, y], t, function(status, result) {
        if (result.bounds) {
            for (var i = 0; i < result.bounds.length; i++) {
                var polygon = new AMap.Polygon({
                    map: map,
                    fillColor: color,
                    fillOpacity: "0.4",
                    strokeColor: color,
                    strokeOpacity: "0.8",
                    strokeWeight: 1
                });
                polygon.setPath(result.bounds[i]);
                polygonArray.push(polygon);
            }
        }
    }, {
        policy: v
    });
}

function addMarkerByAddress(address) {
    var geocoder = new AMap.Geocoder({
        city: "北京",
        radius: 1000
    });
    geocoder.getLocation(address, function(status, result) {
        if (status === "complete" && result.info === 'OK') {
            var geocode = result.geocodes[0];
            rentMarker = new AMap.Marker({
                map: map,
                title: address,
                icon: 'http://webapi.amap.com/theme/v1.3/markers/n/mark_b.png',
                position: [geocode.location.getLng(), geocode.location.getLat()]
            });
            rentMarkerArray.push(rentMarker);

            rentMarker.content = "<div>房源：<a target =
            '_blank' href='http://bj.58.com/pinpaigongyu/?key=" + address + "'>" + address + "</a><div>"
            rentMarker.on('click', function(e) {
                infoWindow.setContent(e.target.content);
                infoWindow.open(map, e.target.getPosition());
                if (amapTransfer) amapTransfer.clear();
                amapTransfer = new AMap.Transfer({
                    map: map,
                    policy: AMap.TransferPolicy.LEAST_TIME,
                    city: "北京市",
                    panel: 'transfer-panel'
                });
                amapTransfer.search([{
                    keyword: workAddress
                }, {
                    keyword: address
                }], function(status, result) {})
            });
        }
    })
}

function delWorkLocation() {
    if (polygonArray) map.remove(polygonArray);
    if (workMarker) map.remove(workMarker);
    polygonArray = [];
}

function delRentLocation()
{
    if (rentMarkerArray) map.remove(rentMarkerArray);
    rentMarkerArray = [];
}

function loadWorkLocation() {
    delWorkLocation();
    var geocoder = new AMap.Geocoder({
        city: "北京",
        radius: 1000
    });

    geocoder.getLocation(workAddress, function(status, result) {
        if (status === "complete" && result.info === 'OK') {
            var geocode = result.geocodes[0];
            x = geocode.location.getLng();
            y = geocode.location.getLat();
            loadWorkMarker(x, y);
            loadWorkRange(x, y, 60, "#3f67a5", vehicle);
            map.setZoomAndCenter(12, [x, y]);
        }
    })
}

function loadRentLocationByFile(fileName) {
    delRentLocation();
    var rent_locations = new Set();
    $.get(fileName, function(data) {
        data = data.split("\n");
        data.forEach(function(item, index)
        {
            rent_locations.add(item.split(",")[1]);
        }
        );
        rent_locations.forEach(function(element, index) {
            addMarkerByAddress(element);
    });
    });
}


        </script>
    </body>
</html>

SyntaxError: invalid syntax (<ipython-input-2-976968218046>, line 1)

* 注：上述代码我运行了，因为不是python语言，会有错误，请忽略！

* 接下来打开cmd,进入到项目文件下index.html所在的地方。在cmd中输入：python -m http.server，然后复制网址：http://127.0.0.1:8000 到火狐浏览器打开，即可看到地图页面,导入我们爬取得房源信息文件，即可看到各个房源在地图上的位置，如图3所示：
<img src="3.png">

* 我们再试试其他的功能，随便点击一个‘小蓝热气球’会发现不仅会自动的给出路径规划，还可以直接点开网址，进入租房网站了解更详细的信息。如图4所示：
<img src="4.png">
<img src="5.png">
<img src="6.png">

# 3、完善
* 项目还有很多有待完善的地方，比如，虽然给出了一小时可以到达的范围，图中类似灰蓝色阴影的区域，但是却把所有的房源信息都标注了，如果能够对房源进行过滤，只显示阴影范围内，也即一小时能够到达的就好了
* 另外，代码的注解大家可以参考实验楼上的帮助大家理解

# 本次的所有代码会放到
[我的博客](https://download.csdn.net/download/fanglichaoliujie/10762034)、[My Github](https://github.com/fangichao/Project/blob/master/RentHouse.rar)进行托管，欢迎访问。如果有什么问题，很乐意进行探讨！